In [1]:
# Connetced yourself Cloud Storage
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

import os
os.chdir("/content/gdrive/My Drive/LATIA112-2/HW2/")
os.listdir()

Mounted at /content/gdrive/


['cpbl.csv', 'requests_books.csv', 'dolls_with_requests.csv', 'hw2.ipynb']

In [5]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500

Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.x75U4icOhl/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.NMLXVm0SB2/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.QkqNb7Ff5k/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

# Selenium 跟 beautiful soup

In [6]:
# 導入Selenium的webdriver模組，用於控制Chrome瀏覽器。
from selenium import webdriver

# 從Selenium函式庫的common.by模組中導入`By`，用於指定元素的定位方式。
from selenium.webdriver.common.by import By

import pandas as pd
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# 建立ChromeOptions物件，這個物件用於設定Chrome瀏覽器的選項。
chrome_options = webdriver.ChromeOptions()

# 使用add_argument()方法添加--headless參數，使Chrome以headless模式運行，即在後台運行，不顯示GUI界面。
chrome_options.add_argument('--headless')

# 使用add_argument()方法添加--no-sandbox參數，這是一個必要的參數，用於解決在某些環境下的運行問題。
chrome_options.add_argument('--no-sandbox')

# 將 headless 屬性設置為 True。
chrome_options.headless = True

# 初始化Chrome WebDriver，並傳遞Chromedriver的路徑和選項。
driver = webdriver.Chrome(options=chrome_options)

# 設置隱式等待時間為10秒。隱式等待是在查找元素時的一種等待方式，
# 如果WebDriver沒有找到元素，它會等待一定的時間再進行查找，直到超過設定的時間為止。
driver.implicitly_wait(10)

# 載入目標網頁
driver.get('https://timtingtravel.com/usa-nyc-broadway-shows/')

# 等待網頁加載完成
driver.implicitly_wait(10)

# 取得網頁內容
html = driver.page_source

# 關閉瀏覽器
driver.quit()

# 使用Beautiful Soup解析HTML
soup = BeautifulSoup(html, 'html.parser')

# 找到表格元素
table = soup.find('table')

# 使用pandas的read_html來解析表格
dfs = pd.read_html(str(table), header=0, keep_default_na=False)

# 選擇第一個表格
selected_table = dfs[0]

# 將資料儲存為CSV檔案
selected_table.to_csv('cpbl.csv', index=False)

print("CSV has Created.")
print(dfs[0])

CSV has Created.
                                                  名稱  \
0  1. 歌劇魅影(The Phantom of the Opera) [已於 2023/4/1...   
1                              2. 獅子王(The Lion King)   
2                                    3. 阿拉丁(Aladdin)   
3    4. 哈利波特與被詛咒的孩子(Harry Potter & the Cursed Child)   
4                                    5. 芝加哥(Chicago)   
5                                6. 綠野仙蹤女巫前傳(Wicked)   
6                                  7. 漢密爾頓(Hamilton)   
7                         8. 摩門經(The Book of Mormon)   
8                        9. 致埃文·漢森(Dear Evan Hansen)   
9                           10. 來自遠方(Come from Away)   

                                                  地點  \
0            Majestic Theatre (莊嚴劇院) [245 W 44th St]   
1          Minskoff Theater(明斯科夫劇場) [200 W 45th St]﻿   
2   New Amsterdam Theatre(新阿姆斯特丹劇院) [214 W 42nd St]﻿   
3               Lyric Theatre(抒情劇院) [214 W 43rd St]﻿   
4          Ambassador Theatre(大使劇場) [219 W 49th St]﻿   
5           Gershwin Theatre(格

# request


In [8]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# 發送GET請求獲取網頁內容
url = 'https://timtingtravel.com/usa-nyc-broadway-shows/'
response = requests.get(url)
html = response.text

# 使用Beautiful Soup解析HTML
soup = BeautifulSoup(html, 'html.parser')

# 找到表格元素
table = soup.find('table')

# 使用pandas的read_html來解析表格
dfs = pd.read_html(str(table), header=0, keep_default_na=False)

# 選擇第一個表格
selected_table = dfs[0]

# 將資料儲存為CSV檔案
selected_table.to_csv('cpb2.csv', index=False)

print("已創建CSV檔案。")
print(dfs[0])


已創建CSV檔案。
                                                  名稱  \
0  1. 歌劇魅影(The Phantom of the Opera) [已於 2023/4/1...   
1                              2. 獅子王(The Lion King)   
2                                    3. 阿拉丁(Aladdin)   
3    4. 哈利波特與被詛咒的孩子(Harry Potter & the Cursed Child)   
4                                    5. 芝加哥(Chicago)   
5                                6. 綠野仙蹤女巫前傳(Wicked)   
6                                  7. 漢密爾頓(Hamilton)   
7                         8. 摩門經(The Book of Mormon)   
8                        9. 致埃文·漢森(Dear Evan Hansen)   
9                           10. 來自遠方(Come from Away)   

                                                  地點  \
0            Majestic Theatre (莊嚴劇院) [245 W 44th St]   
1          Minskoff Theater(明斯科夫劇場) [200 W 45th St]﻿   
2   New Amsterdam Theatre(新阿姆斯特丹劇院) [214 W 42nd St]﻿   
3               Lyric Theatre(抒情劇院) [214 W 43rd St]﻿   
4          Ambassador Theatre(大使劇場) [219 W 49th St]﻿   
5           Gershwin Theatre(格甚溫劇院) [